# IMPORTS

In [ ]:
from google.colab import drive
from tqdm import tqdm
import torch
from torchmetrics.detection import MeanAveragePrecision
import os
import cv2
import shutil
import time
import zipfile
from collections import defaultdict
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pandas as pd
import random
import seaborn as sns
import pickle
import glob
import csv
from collections import Counter

In [ ]:
drive.mount("/content/drive", force_remount=True)

# FUNCTIONS

In [ ]:
def load_detections(pkl_path):
    """Loads model prediction dictionary from a pickle file."""
    try:
        # Open the pickle file to get the stored outputs
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            # Detections are stored under the 'outputs' key
            return data.get('outputs', {})
    except (FileNotFoundError, pickle.UnpicklingError, KeyError) as e:
        # Handle cases where the file is missing, corrupt, or has the wrong format
        print(f"Error loading detections from {pkl_path}: {e}")
        return {}

In [ ]:
def visualize_bin_collage(bin_df, bin_label, metric_col, data, metric_name=None, num_images=3, cols=3, title_prefix="Bin", target_size=1024):
    """Creates a collage of sample images from a specific data bin."""

    # Take a random, consistent sample of images from the provided dataframe
    sample_df = bin_df.sample(min(num_images, len(bin_df)), random_state=42)

    # Set up the plot grid
    rows = (len(sample_df) + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(5 * cols, 5 * rows))
    axes = axes.flatten()

    # Default to using the metric column name if a display name isn't provided
    metric_name = metric_name or metric_col

    # Hide any unused subplots in the grid
    for ax in axes[len(sample_df):]:
        ax.axis('off')

    # Loop through the sampled images and plot them
    for i, (_, row) in enumerate(sample_df.iterrows()):
        img_name = os.path.basename(row['image_name'])
        dataset_name = row['original_dataset']
        metric_val = row[metric_col]

        # Get metadata for this image (path, GT boxes, etc.)
        info = get_dataset_info_by_name(img_name, data)
        if not info:
            print(f"Unknown dataset for image: {img_name}")
            continue

        try:
            # Load, resize, and display the image
            img = Image.open(info['path']).convert("RGB").resize((target_size, target_size))
            ax = axes[i]
            ax.imshow(np.array(img))
            ax.axis('off')
            ax.set_title(f"{img_name}\n{metric_name}: {metric_val:.1f} | {dataset_name}", fontsize=10)

            # Scale the ground truth boxes to match the resized image
            scaled_boxes = scale_boxes(info['gt_boxes'], info['width'], info['height'], target_size, target_size)

            # Draw the ground truth boxes on the image
            for box in scaled_boxes:
                x1, y1, x2, y2 = box
                rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2, edgecolor='lime', facecolor='none')
                ax.add_patch(rect)
        except FileNotFoundError:
            # Handle cases where the image file is missing
            print(f"Image not found at path: {info['path']}")
            axes[i].set_title(f"{img_name}\n(Image not found)", fontsize=10)
            axes[i].axis('off')

    # Add a main title to the entire collage
    plt.suptitle(f"{title_prefix}: {bin_label}", fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()

In [ ]:
def get_dataset_info_by_name(img_name_cleaned, data):
    """
    Looks up an image's metadata (path, GT boxes, dimensions)
    by searching across all loaded datasets.
    """
    # Loop through each dataset config (synthetic, nominal, edge)
    for dtype, config in DATASET_CONFIGS.items():
        df = data[dtype]['df']

        # Check if the image name exists in this dataset's dataframe
        # Using .values is a quick way to search the series
        if img_name_cleaned in df['image_name'].values:
            # If found, get the corresponding row for this image
            row = df[df['image_name'] == img_name_cleaned].iloc[0]

            # Return a dictionary with all the necessary info
            return {
                "img_path": os.path.join(config['image_folder'], img_name_cleaned),
                "gt_boxes": data[dtype]['gt'].get(img_name_cleaned, []),
                "width": row['width'],
                "height": row['height']
            }

    # If the loop finishes without finding the image, return None
    return None

In [ ]:
def compute_quad_area(row):
    """Calculates the area of a quadrilateral from its corner coordinates."""

    # Unpack the four corner points (A, B, C, D) from the input row
    x1, y1 = row['x_A'], row['y_A']
    x2, y2 = row['x_B'], row['y_B']
    x3, y3 = row['x_C'], row['y_C']
    x4, y4 = row['x_D'], row['y_D']

    # Apply the Shoelace formula for the area of a simple polygon
    return 0.5 * abs(
        x1*y2 + x2*y3 + x3*y4 + x4*y1
        - y1*x2 - y2*x3 - y3*x4 - y4*x1
    )

In [ ]:
def get_multiple_detections(outputs, score_thresh=0.90, detailed=False):
    """
    Finds images that have more than one detection above a certain score threshold.
    This is useful for identifying duplicate detections.
    """
    results = []

    # Go through each image's predictions in the outputs dictionary
    for img_name, pred in outputs.items():
        scores = pred['scores']
        boxes = pred['boxes']

        # Create a boolean mask to find predictions with high enough scores
        high_score_mask = scores >= score_thresh
        high_score_boxes = boxes[high_score_mask]
        high_scores = scores[high_score_mask]

        # If more than one high-confidence box was found, it's a case of multiple detections
        if len(high_score_boxes) > 1:
            # The 'detailed' flag controls how much info we save
            if detailed:
                # Save the image name along with the actual boxes and scores
                results.append((img_name, high_score_boxes, high_scores))
            else:
                # Otherwise, just save the count of how many extra boxes there were
                results.append((img_name, len(high_score_boxes)))

    # Return the list of images that met the criteria, and the total count
    return results, len(results)

In [ ]:
def load_ground_truth(gt_folder, image_extension="png"):
    """
    Loads ground truth bounding boxes from text files in a specified folder.
    Each text file corresponds to an image and contains bounding box data.
    """
    # Make sure the image extension has a leading dot
    if not image_extension.startswith("."):
        image_extension = "." + image_extension

    gt_data = {}
    # Find all .txt files in the ground truth folder
    txt_files = glob.glob(os.path.join(gt_folder, "*.txt"))

    # Process each ground truth file
    for txt_file in txt_files:
        # Infer the corresponding image name from the text file's name
        image_name, _ = os.path.splitext(os.path.basename(txt_file))
        image_name += image_extension

        boxes = []
        try:
            with open(txt_file, 'r') as f:
                for line in f:
                    # Expecting format: class_id x1 y1 x2 y2
                    parts = line.strip().split()
                    if len(parts) == 5:
                        try:
                            # Unpack the coordinates, ignoring the class_id
                            _, x1, y1, x2, y2 = map(float, parts)
                            # Standardize the box format to (xmin, ymin, xmax, ymax)
                            boxes.append([min(x1, x2), min(y1, y2), max(x1, x2), max(y1, y2)])
                        except ValueError:
                            # Skip lines with non-numeric coordinate values
                            pass
            if boxes:
                # Map the list of boxes to the corresponding image name
                gt_data[image_name] = boxes
        except Exception as e:
            print(f"Error reading GT file {txt_file}: {e}")

    return gt_data

In [ ]:
def scale_boxes(boxes, orig_w, orig_h, target_w, target_h):
    """
    Scales bounding boxes from an original image size to a new target size.
    Useful for converting between original image coordinates and model input coordinates.
    """
    # Avoid division by zero if original dimensions are invalid
    if orig_w == 0 or orig_h == 0:
        return []

    # Calculate the scaling factor for width and height
    x_scale = target_w / orig_w
    y_scale = target_h / orig_h

    # Apply the scaling factor to each coordinate of each box
    return [[b[0] * x_scale, b[1] * y_scale, b[2] * x_scale, b[3] * y_scale] for b in boxes]

In [ ]:
def draw_predictions_with_gt(img_path, gt_boxes_orig, pred_boxes, pred_scores, orig_w, orig_h, ax, dataset_name, score_threshold=0.90, target_size=1024):
    """
    Draws ground truth boxes and the model's filtered predictions on an image.
    """
    # Load and resize the image for consistent display
    img = Image.open(img_path).convert("RGB").resize((target_size, target_size))
    ax.imshow(img)
    ax.axis('off')
    ax.set_title(f"{os.path.basename(img_path)}\n{dataset_name}", fontsize=9)

    # Draw Ground Truth boxes in green
    if gt_boxes_orig:
        # Scale GT boxes from original image coordinates to the displayed size
        gt_boxes_scaled = scale_boxes(gt_boxes_orig, orig_w, orig_h, target_size, target_size)
        for box in gt_boxes_scaled:
            x1, y1, x2, y2 = box
            rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2, edgecolor='lime', facecolor='none')
            ax.add_patch(rect)

    # Filter and draw the model's prediction boxes in red
    for i, box in enumerate(pred_boxes):
        # Only draw boxes that are above the confidence score threshold
        if pred_scores[i] >= score_threshold:
            x1, y1, x2, y2 = box
            rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=1, linestyle='--', edgecolor='red', facecolor='none')
            ax.add_patch(rect)
            # Add the confidence score as a text label on the box
            ax.text(x1, y1 - 5, f'{pred_scores[i]:.2f}', bbox=dict(facecolor='red', alpha=0.5), fontsize=7, color='white')

In [ ]:
def tag_with_dataset(multi_list, tag):
    """Appends a dataset tag to each item in a list of detections."""

    # Uses a list comprehension to iterate through the list and add the tag
    # Input format: [(img, boxes, scores), ...]
    # Output format: [(img, boxes, scores, tag), ...]
    return [(img, boxes, scores, tag) for (img, boxes, scores) in multi_list]

In [ ]:
def compute_iou(box1, box2):
    """Calculates the Intersection over Union (IoU) for two bounding boxes."""

    # box format is [x1, y1, x2, y2]

    # Determine the coordinates of the intersection rectangle
    xA = max(box1[0], box2[0])
    yA = max(box1[1], box2[1])
    xB = min(box1[2], box2[2])
    yB = min(box1[3], box2[3])

    # Compute the area of intersection, adding 1 for pixel-based coordinates
    # The max(0, ...) ensures the area is non-negative if boxes don't overlap
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    # Compute the area of both the prediction and ground-truth rectangles
    box1Area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2Area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    # Calculate the IoU by dividing the intersection area by the union area
    # Union Area = Box1 Area + Box2 Area - Intersection Area
    iou = interArea / float(box1Area + box2Area - interArea)

    return iou

In [ ]:
def evaluate_dataset(dataset_type, iou_threshold=0.3, score_threshold=0.90, return_detailed=False):
    """
    Calculates performance metrics (TP, FP, FN, etc.) for a given dataset.
    Can also return a detailed, per-image breakdown of results.
    """
    # Grab all the necessary data for this dataset from our main data dictionary
    dataset_info = data[dataset_type]
    gt_data = dataset_info['gt']
    predictions = dataset_info['outputs']
    df = dataset_info['df']
    # Create a quick lookup for image dimensions for scaling
    image_dims_lookup = dict(zip(df['image_name'], zip(df['width'], df['height'])))

    # Initialize counters for the evaluation metrics
    TP, FP, FN = 0, 0, 0
    detailed_results = []

    # Get a set of all unique image names across both ground truth and predictions
    all_image_names = set(gt_data.keys()) | set(predictions.keys())

    # Go through each image to evaluate its predictions
    for img_name in all_image_names:
        # Get ground truth and predictions for the current image
        gt_boxes_orig = gt_data.get(img_name, [])
        pred_dict = predictions.get(img_name, {})
        pred_scores = pred_dict.get('scores', [])
        pred_boxes = pred_dict.get('boxes', [])

        # Filter predictions to only include those above the score threshold
        high_score_boxes = [box for i, box in enumerate(pred_boxes) if pred_scores[i] >= score_threshold]

        # Store detailed results for this image, assuming all preds are FP and all GTs are FN initially
        image_result = {
            'image_name': img_name,
            'dataset_name': df[df['image_name'] == img_name]['original_dataset'].iloc[0] if not df[df['image_name'] == img_name].empty else 'Unknown',
            'predicted_boxes': high_score_boxes,
            'ground_truth_boxes': gt_boxes_orig,
            'is_predicted_box_tp': [False] * len(high_score_boxes),
            'is_gt_box_fn': [True] * len(gt_boxes_orig)
        }

        orig_w, orig_h = image_dims_lookup.get(img_name, (0, 0))

        # Scale GT boxes to the model's coordinate space (1024x1024) for comparison
        scaled_gt_boxes = scale_boxes(gt_boxes_orig, orig_w, orig_h, 1024, 1024)

        # Keep track of which GT boxes have already been matched to a prediction
        matched_gt_indices = set()
        # Iterate through each prediction to find a matching GT box
        for pred_idx, pred_box in enumerate(high_score_boxes):
            best_iou, best_gt_idx = -1, -1
            # Find the best GT box match for the current prediction
            for gt_idx, gt_box in enumerate(scaled_gt_boxes):
                iou = compute_iou(pred_box, gt_box)
                if iou > best_iou:
                    best_iou, best_gt_idx = iou, gt_idx

            # If a good, unmatched GT box is found, count it as a True Positive
            if best_iou >= iou_threshold and best_gt_idx not in matched_gt_indices:
                TP += 1
                matched_gt_indices.add(best_gt_idx)
                image_result['is_predicted_box_tp'][pred_idx] = True  # Mark prediction as TP
                if best_gt_idx < len(image_result['is_gt_box_fn']):
                    image_result['is_gt_box_fn'][best_gt_idx] = False # Mark GT as "found"
            else:
                # Otherwise, the prediction is a False Positive
                FP += 1

        # Any GT boxes that were never matched are False Negatives
        FN += len(scaled_gt_boxes) - len(matched_gt_indices)

        # Store the detailed breakdown if the user requested it
        if return_detailed:
            detailed_results.append(image_result)

    if return_detailed:
        return detailed_results

    # Calculate final precision, recall, and F1 score
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else

In [ ]:
def run_sensitivity_analysis(sweep_param, fixed_param_value, sweep_range):
    """
    Runs a sensitivity analysis for a given parameter (IoU or score) across all datasets.
    This helps find the optimal thresholds by testing a range of values.
    """
    all_results = []

    # Loop through each dataset (synthetic, nominal, edge)
    for dataset_type, config in DATASET_CONFIGS.items():
        print(f"Running {sweep_param} sweep for {config['name']}...")
        # Loop through the provided range of threshold values
        for threshold_val in sweep_range:

            # Call the main evaluation function with the correct parameter being swept
            if sweep_param == 'iou':
                # Sweep IoU, hold score constant
                result = evaluate_dataset(dataset_type, iou_threshold=threshold_val, score_threshold=fixed_param_value)
            elif sweep_param == 'score':
                # Sweep score, hold IoU constant
                result = evaluate_dataset(dataset_type, iou_threshold=fixed_param_value, score_threshold=threshold_val)
            else:
                raise ValueError("sweep_param must be 'iou' or 'score'")

            # Store the results in a structured list for later analysis
            all_results.append({
                'dataset': config['name'],
                'sweep_param_value': threshold_val,
                'precision': result['precision'],
                'recall': result['recall'],
                'f1_score': result['f1_score']
            })

    # Return a single pandas DataFrame containing all the collected results
    return pd.DataFrame(all_results)

In [ ]:
def plot_sensitivity_results(results_df, sweep_param):
    """
    Plots the results of a sensitivity analysis (e.g., precision, recall vs. threshold)
    for multiple datasets in a single grid.
    """
    # Reshape the DataFrame from a wide to a long format.
    # This is necessary to use the different metrics (precision, recall, f1) as a 'hue' in seaborn.
    df_melted = results_df.melt(
        id_vars=['dataset', 'sweep_param_value'],
        value_vars=['precision', 'recall', 'f1_score'],
        var_name='metric',
        value_name='score'
    )

    # Set up a FacetGrid, which creates a separate plot column for each dataset
    g = sns.FacetGrid(df_melted, col="dataset", hue="metric", col_wrap=2, height=5)

    # Map the lineplot function onto each subplot in the grid
    g.map(sns.lineplot, "sweep_param_value", "score", marker='o').add_legend()

    # Set the axis labels and titles for clarity
    g.set_axis_labels(f'{sweep_param.capitalize()} Threshold', 'Score')
    g.set_titles("{col_name}") # Use the dataset name as the subplot title
    plt.ylim(-0.05, 1.05)
    plt.tight_layout()
    plt.show()

# LOADING FILES

In [ ]:
DATASET_CONFIGS = {
    'synthetic': {
        'csv_path': "/content/drive/MyDrive/Dataset/synthetic_with_detection_flags.csv",
        'pkl_path': '/content/drive/MyDrive/test_outputs_synthetic.pkl',
        'image_folder': "/content/drive/MyDrive/Dataset/data_tlbr/TestSynth/images",
        'gt_folder': '/content/drive/MyDrive/Dataset/data_tlbr/TestSynth/labels',
        'image_ext': ".jpeg",
        'name': 'LARD_test_synth'
    },
    'nominal': {
        'csv_path': "/content/drive/MyDrive/Dataset/real_nominal_with_detection_flags.csv",
        'pkl_path': '/content/drive/MyDrive/test_outputs_real_nominal.pkl',
        'image_folder': "/content/drive/MyDrive/Dataset/data_tlbr/Test_RealNominal/images",
        'gt_folder': '/content/drive/MyDrive/Dataset/data_tlbr/Test_RealNominal/labels',
        'image_ext': ".png",
        'name': 'REAL_Nominal'
    },
    'edge': {
        'csv_path': "/content/drive/MyDrive/Dataset/real_edge_with_detection_flags.csv",
        'pkl_path': '/content/drive/MyDrive/test_outputs_real_edge.pkl',
        'image_folder': "/content/drive/MyDrive/Dataset/data_tlbr/Test_RealEdge/images",
        'gt_folder': '/content/drive/MyDrive/Dataset/data_tlbr/Test_RealEdge/labels',
        'image_ext': ".png",
        'name': 'REAL_Edge_Cases'
    }
}

In [ ]:
data = {}
for dtype, config in DATASET_CONFIGS.items():
    print(f"Loading {dtype} data...")
    data[dtype] = {
        'df': pd.read_csv(config['csv_path']),
        'outputs': load_detections(config['pkl_path']),
        'gt': load_ground_truth(config['gt_folder'], config['image_ext'])
    }
print("All data loaded successfully.")

# Threshold Analysis

In [ ]:
# Collect all prediction scores from every image across all datasets
all_scores = []
for dataset_type in DATASET_CONFIGS:
    outputs = data[dataset_type]['outputs']
    for img_name, preds in outputs.items():
        all_scores.extend(preds.get('scores', []))

# Convert the list of scores into a pandas DataFrame for easy analysis
scores_df = pd.DataFrame(all_scores, columns=['score'])

# Create the distribution plot using a histogram
plt.figure(figsize=(12, 6))
sns.histplot(data=scores_df, x='score', bins=50, kde=True)
plt.title('Distribution of All Prediction Confidence Scores', fontsize=16)
plt.xlabel('Confidence Score')
plt.ylabel('Number of Detections')
# Add a vertical line to mark the start of our analysis range
plt.axvline(x=0.90, color='r', linestyle='--', linewidth=2, label='Analysis Range Start (0.90)')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Print summary statistics to quantify the score distribution
print(f"Total number of detections (all scores): {len(scores_df)}")
print(f"Number of detections with score >= 0.90: {len(scores_df[scores_df['score'] >= 0.90])}")
high_score_percentage = (len(scores_df[scores_df['score'] >= 0.90]) / len(scores_df)) * 100
print(f"Percentage of detections with score >= 0.90: {high_score_percentage:.2f}%")

In [ ]:
# Define the different IoU thresholds and score thresholds we want to test
iou_thresholds = [0.3, 0.5, 0.7]
score_thresholds_to_test = np.arange(0.90, 1.00, 0.01)
comprehensive_results = []

# Run the evaluation in a nested loop to test every combination
for iou in iou_thresholds:
    print(f"--- Running analysis for IoU Threshold: {iou} ---")
    # Loop through each dataset
    for dataset_type, config in DATASET_CONFIGS.items():
        # Loop through each score threshold
        for score in score_thresholds_to_test:
            # Call our main evaluation function with the current combination of thresholds
            result = evaluate_dataset(dataset_type, iou_threshold=iou, score_threshold=score)

            # Store the results in a list of dictionaries for easy conversion to a DataFrame
            comprehensive_results.append({
                'iou_threshold': iou,
                'score_threshold': score,
                'dataset': config['name'],
                'precision': float(result['precision']), # Make sure data is numeric for plotting
                'recall': float(result['recall']),
                'f1_score': float(result['f1_score'])
            })

# Convert the list of results into a single pandas DataFrame
results_df = pd.DataFrame(comprehensive_results)
print("\nComprehensive analysis complete.")
print(results_df.head())

In [ ]:
print("\n--- Generating Precision-Recall Curves ---")

# Use FacetGrid to create a separate plot for each dataset
g_pr = sns.FacetGrid(results_df, col="dataset", hue="iou_threshold", height=5, palette='viridis')

# Map a line plot onto the grid
g_pr.map(sns.lineplot, "recall", "precision", marker='.', markersize=10).add_legend()

# Set labels and titles
g_pr.set_axis_labels("Recall", "Precision")
g_pr.set_titles("Dataset: {col_name}")
fig = g_pr.fig
fig.suptitle("Precision-Recall Curve Analysis", y=1.03, fontsize=16) # Add a main title
plt.tight_layout()
plt.show()

In [ ]:
print("\n--- Generating F1 Score vs. Confidence Threshold Curves (Highlighting Best Overall) ---")

# Create the main plot grid using seaborn's relplot for a figure-level plot
g_f1 = sns.relplot(
    data=results_df,
    x="score_threshold",
    y="f1_score",
    hue="iou_threshold", # Color lines by IoU threshold
    col="dataset",       # Create a separate column for each dataset
    kind="line",         # Specify a line plot
    marker='.',
    markersize=10,
    height=5,
    aspect=1.2,
    palette='viridis',
    legend='full'
)

# Find the single best operating point (highest F1 score) for each dataset
# idxmax() finds the index of the first occurrence of the maximum value
idx = results_df.groupby(['dataset'])['f1_score'].idxmax()
best_overall_points = results_df.loc[idx]


# Manually iterate through each subplot (ax) and plot a star for the best point
# This is a robust way to add annotations to a FacetGrid
for ax, dataset_name in zip(g_f1.axes.flat, g_f1.col_names):

    # Find the row in our 'best_overall_points' DataFrame that matches this subplot's dataset
    point_to_plot = best_overall_points[best_overall_points['dataset'] == dataset_name]

    # If we found a point, plot a single large star on this subplot
    if not point_to_plot.empty:
        # Get the coordinates for the star from our best_overall_points df
        x = point_to_plot['score_threshold'].iloc[0]
        y = point_to_plot['f1_score'].iloc[0]

        # Use matplotlib's scatter to draw the star on the current axis
        ax.scatter(x, y, marker='*', s=500, color='gold', edgecolor='black', zorder=10)


# Add final labels and a main title to the figure
g_f1.set_axis_labels("Confidence Score Threshold", "F1 Score")
g_f1.fig.suptitle("F1 Score vs. Confidence Threshold (Overall Best Point Highlighted)", y=1.03, fontsize=16)
plt.tight_layout()
plt.show()


# Print the summary table showing the exact values for the best operating points
print("\n--- Optimal Operating Point for Each Dataset ---")
print(best_overall_points[['dataset', 'iou_threshold', 'score_threshold', 'f1_score', 'precision', 'recall']])

# Results per dataset

In [ ]:
# Instantiate the mAP metric from torchmetrics, specifying the box format
metric = MeanAveragePrecision(box_format='xyxy')

# Create a dictionary to store the final results for each dataset
all_map_results = {}

# Loop through each dataset to calculate its mAP separately
for dataset_type, config in DATASET_CONFIGS.items():
    dataset_name = config['name']
    print(f"--- Calculating mAP for: {dataset_name} ---")

    # Get the pre-loaded ground truth, predictions, and dataframe for this dataset
    gt_data = data[dataset_type]['gt']
    predictions = data[dataset_type]['outputs']
    df = data[dataset_type]['df']

    # Create a quick lookup for original image dimensions for scaling
    image_dims_lookup = dict(zip(df['image_name'], zip(df['width'], df['height'])))

    # These lists will hold the formatted data required by torchmetrics
    preds_for_metric = []
    targets_for_metric = []

    # Iterate through every image that has a ground truth label
    for image_name, gt_boxes in gt_data.items():
        # Format the ground truth (target) boxes into the required tensor format
        targets_for_metric.append({
            'boxes': torch.tensor(gt_boxes, dtype=torch.float32),
            'labels': torch.ones(len(gt_boxes), dtype=torch.int32) # Assign a single class label
        })

        # Get the predictions for this image, defaulting to empty if none exist
        pred_dict = predictions.get(image_name, {})
        pred_boxes_model_space = pred_dict.get('boxes', [])
        pred_scores = pred_dict.get('scores', [])

        # Get the original image dimensions to scale the predictions
        orig_w, orig_h = image_dims_lookup.get(image_name, (0, 0))
        if orig_w == 0 or orig_h == 0:
            # If dimensions are missing, we can't scale, so use empty boxes
            pred_boxes_orig_space = []
        else:
            # Scale prediction boxes from model space (1024x1024) back to original image space
            pred_boxes_orig_space = scale_boxes(pred_boxes_model_space, 1024, 1024, orig_w, orig_h)

        # Format the scaled predictions into the required tensor format
        preds_for_metric.append({
            'boxes': torch.tensor(pred_boxes_orig_space, dtype=torch.float32),
            'scores': torch.tensor(pred_scores, dtype=torch.float32),
            'labels': torch.ones(len(pred_boxes_orig_space), dtype=torch.int32)
        })

    # Update the metric with all the predictions and targets for this dataset
    metric.update(preds_for_metric, targets_for_metric)
    # Compute the final mAP scores
    results = metric.compute()
    all_map_results[dataset_name] = results
    # Reset the metric to clear its state before processing the next dataset
    metric.reset()

# Print a summary table of the final results
print("\n\n--- mAP Evaluation Summary ---")
summary_data = []
for dataset_name, metrics in all_map_results.items():
    summary_data.append({
        'Dataset': dataset_name,
        'mAP': f"{metrics['map'].item():.4f}",
        'mAP_50': f"{metrics['map_50'].item():.4f}",
        'mAP_75': f"{metrics['map_75'].item():.4f}",
        'mAP (small)': f"{metrics['map_small'].item():.4f}",
        'mAP (medium)': f"{metrics['map_medium'].item():.4f}",
        'mAP (large)': f"{metrics['map_large'].item():.4f}",
    })

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

# DETECTION RATES

In [ ]:
print("--- Starting to update CSV files with 'detected' column ---")

# Define the confidence score threshold for a detection to be considered valid
SCORE_THRESHOLD = 0.9

# Loop through each dataset defined in our main config
for dataset_type, config in DATASET_CONFIGS.items():
    dataset_name = config['name']
    csv_path = config['csv_path']
    print(f"\nProcessing dataset: {dataset_name}...")

    # Get the DataFrame and model predictions for the current dataset
    df = data[dataset_type]['df']
    outputs = data[dataset_type]['outputs']

    # Create a lookup map of image_name -> True/False based on detection scores
    detection_status = {}
    for image_name, predictions in outputs.items():
        # Check if any score in this image's prediction array is >= our threshold
        if np.any(predictions.get('scores', []) >= SCORE_THRESHOLD):
            detection_status[image_name] = True
        else:
            detection_status[image_name] = False

    # Create the 'detected' column by mapping the status to each image name
    # .fillna(False) handles images that are in the CSV but had no model output
    df['detected'] = df['image_name'].map(detection_status).fillna(False)

    # Save the modified DataFrame back to its original CSV file path
    try:
        df.to_csv(csv_path, index=False)
        print(f"Successfully updated and saved: {csv_path}")
    except Exception as e:
        print(f"❌ Failed to save {csv_path}. Error: {e}")

print("\n--- All CSV files have been processed. ---")

In [ ]:
print("Calculating detection rates for all datasets...")

for dataset_type, config in DATASET_CONFIGS.items():
    # Access the DataFrame from our central 'data' dictionary
    df = data[dataset_type]['df']

    # Check if the 'detected' column exists to avoid errors
    if 'detected' in df.columns:
        detection_rate = df['detected'].mean()
        # Use the 'name' from our config for a nice, readable label
        dataset_name = config.get('name', dataset_type.capitalize())
        print(f"Detection Rate ({dataset_name}): {detection_rate:.2%}")
    else:
        print(f"Warning: 'detected' column not found in '{dataset_type}' dataset.")

## Detection Rate vs Slant Distance (Synthetic only)

In [ ]:
df_analysis = data['synthetic']['df'].copy()

# The rest of your code works perfectly with this new df_analysis
df_analysis["slant_distance"] = pd.to_numeric(df_analysis["slant_distance"], errors='coerce')
df_analysis.dropna(subset=["slant_distance"], inplace=True)

# Bin slant distance
num_bins = 10
df_analysis["slant_bin"] = pd.cut(df_analysis["slant_distance"], bins=num_bins)

# Group by bin and compute detection rate
bin_stats = df_analysis.groupby("slant_bin", observed=False)["detected"].agg(["mean", "count"]).reset_index()
bin_stats.rename(columns={"mean": "detection_rate", "count": "num_images"}, inplace=True)

# Plot
plt.figure(figsize=(10, 5))
sns.barplot(data=bin_stats, x="slant_bin", y="detection_rate")
plt.xticks(rotation=45, ha='right')
plt.xlabel("Slant Distance Bin (Km)")
plt.ylabel("Detection Rate")

title_name = DATASET_CONFIGS['synthetic']['name']
plt.title(f"Detection Rate vs Slant Distance ({title_name})")

plt.grid(axis="y", linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 4))
sns.lineplot(data=bin_stats, x=bin_stats.index, y="detection_rate", marker='o')
plt.xticks(ticks=bin_stats.index, labels=[str(b) for b in bin_stats["slant_bin"]], rotation=45, ha='right')
plt.title("Detection Rate Trend across Slant Distance")
plt.xlabel("Slant Distance Bin")
plt.ylabel("Detection Rate")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Get the unique bins we created in the previous step from the dataframe
slant_bins = df_analysis["slant_bin"].unique()

# Sort the bins by their lower edge for a clean, ordered visualization
slant_bins = sorted(slant_bins, key=lambda x: x.left)

print("--- Visualizing examples from a few slant distance bins ---")
# Loop through a sample of the bins to visualize them (e.g., every 2nd bin)
for bin_label in slant_bins[::2]:
    # Create a temporary dataframe containing only the images from the current bin
    bin_df = df_analysis[df_analysis["slant_bin"] == bin_label]
    if bin_df.empty:
        continue

    print(f"\n📸 Visualizing Slant Distance Bin: {bin_label}")

    # Call our main collage function to display a sample of images from this bin
    visualize_bin_collage(
        bin_df,
        bin_label=bin_label,
        metric_col="slant_distance",
        metric_name="Slant Distance (km)",
        title_prefix="Slant Distance Bin",
        data=data,  # Pass the main data dictionary for metadata lookup
        num_images=3,
        cols=3
    )

## Detection Rate vs Area of runway (in pixels)

In [ ]:
print("Computing runway area for all datasets...")
for dataset_type in data:
    # Get the dataframe from the dictionary
    df = data[dataset_type]['df']
    df['runway_area'] = df.apply(compute_quad_area, axis=1)

# 2. MODIFIED CONCATENATION: Create the list of DataFrames from the 'data' dictionary.
list_of_dfs = [data[dataset_type]['df'] for dataset_type in data]
all_df = pd.concat(list_of_dfs, ignore_index=True)

In [ ]:
print("Binning and plotting results...")
# Bin into quantile-based buckets across all datasets
all_df['area_bin'] = pd.qcut(all_df['runway_area'], q=5, duplicates='drop')

# Group by area bin and source
area_stats = all_df.groupby(['area_bin', 'original_dataset'], observed=False)['detected'].mean().reset_index()

# Plotting
plt.figure(figsize=(12, 6))
sns.barplot(data=area_stats, x='area_bin', y='detected', hue='original_dataset')
plt.title('Detection Rate vs. Runway Area (Quantile Bins)')
plt.xlabel('Runway Area (Binned)')
plt.ylabel('Detection Rate')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
area_bins = all_df['area_bin'].cat.categories
bins_to_visualize = [area_bins[0], area_bins[2], area_bins[4]]

for bin_label in bins_to_visualize:
    bin_df = all_df[all_df['area_bin'] == bin_label]
    if bin_df.empty:
        continue
    visualize_bin_collage(
        bin_df,
        bin_label,
        metric_col="runway_area",
        metric_name="Area",
        title_prefix="Runway Area Bin",
        data=data # Pass the data dictionary here
    )

## Detection Rate vs Time to Landing (TTL)

In [ ]:
df_real = pd.concat([data['nominal']['df'], data['edge']['df']], ignore_index=True)

# Drop rows with missing TTL or detection
df_real = df_real.dropna(subset=['time_to_landing', 'detected'])

# Convert to numeric
df_real['time_to_landing'] = pd.to_numeric(df_real['time_to_landing'])

In [ ]:
# Define the width (in seconds) for each time-to-landing bin
bin_width = 6

# Find the maximum TTL value in the dataset to determine the upper range for our bins
max_ttl = df_real['time_to_landing'].max()

# Create a list of bin edges, starting from 0 up to the max TTL, in steps of bin_width
# We add an extra bin at the end to ensure the max value is included.
bins = list(range(0, int(max_ttl) + bin_width * 2, bin_width))

# Create corresponding labels for each bin (e.g., "0-6", "6-12")
labels = [f"{b}-{b+bin_width}" for b in bins[:-1]]

# Use pandas.cut to categorize each row's TTL value into the appropriate bin
df_real['ttl_bin'] = pd.cut(df_real['time_to_landing'], bins=bins, labels=labels, right=False)

In [ ]:
# Group by the TTL bin and the original dataset to calculate the mean detection rate for each subgroup
ttl_stats = df_real.groupby(['ttl_bin', 'original_dataset'], observed=True)['detected'].mean().reset_index()

# Group by the same categories to get the number of images in each subgroup
ttl_counts = df_real.groupby(['ttl_bin', 'original_dataset'], observed=True).size().reset_index(name='count')

In [ ]:
# Group by TTL bin to get the image count for the Nominal dataset
ttl_bin_counts_nominal = df_real[df_real['original_dataset'] == 'REAL_Nominal'].groupby(['ttl_bin'], observed=True).size().reset_index(name='count')
# Do the same for the Edge Cases dataset
ttl_bin_counts_edge = df_real[df_real['original_dataset'] == 'REAL_Edge_Cases'].groupby(['ttl_bin'], observed=True).size().reset_index(name='count')

# Merge the two count dataframes into a single table for comparison
combined_ttl_counts = pd.merge(
    ttl_bin_counts_nominal,
    ttl_bin_counts_edge,
    on='ttl_bin',
    suffixes=('_nominal', '_edge'),
    how='outer' # Use an outer join to make sure no bins are dropped
)

# Clean up the merged data by filling any missing values (NaN) with 0
combined_ttl_counts[['count_nominal', 'count_edge']] = combined_ttl_counts[['count_nominal', 'count_edge']].fillna(0)

# Convert the count columns to integers for a cleaner look
combined_ttl_counts['count_nominal'] = combined_ttl_counts['count_nominal'].astype(int)
combined_ttl_counts['count_edge'] = combined_ttl_counts['count_edge'].astype(int)


# Calculate the total number of images per bin
combined_ttl_counts['total_count'] = combined_ttl_counts['count_nominal'] + combined_ttl_counts['count_edge']

# Rename columns for better readability in the final table
combined_ttl_counts.rename(columns={
    'count_nominal': 'Real-Nominal Count',
    'count_edge': 'Real-Edge Count'
}, inplace=True)

# Sort the table by the TTL bins to ensure it's in chronological order
# This is important because 'pd.cut' creates a categorical type that has a defined order
combined_ttl_counts['ttl_bin'] = pd.Categorical(
    combined_ttl_counts['ttl_bin'],
    categories=df_real['ttl_bin'].cat.categories,
    ordered=True
)
combined_ttl_counts = combined_ttl_counts.sort_values('ttl_bin')


# Display the final, formatted table
print(combined_ttl_counts.to_string(index=False))

In [ ]:
ttl_stats_filtered = ttl_stats[ttl_stats['ttl_bin'].isin(combined_ttl_counts['ttl_bin'])]

In [ ]:
fig, (ax_plot, ax_table) = plt.subplots(
    1, 2,
    figsize=(16, 8),
    gridspec_kw={'width_ratios': [3, 1]}  # Wider plot, narrower table
)

# Bar Plot (left)
default_palette = sns.color_palette()
palette = {
    'REAL_Nominal': default_palette[1],
    'REAL_Edge_Cases': default_palette[0]
}

sns.barplot(
    data=ttl_stats_filtered,
    x='ttl_bin',
    y='detected',
    hue='original_dataset',
    hue_order=['REAL_Nominal', 'REAL_Edge_Cases'],
    palette=palette,
    ax=ax_plot
)

ax_plot.set_xlabel('Time to Landing (seconds)')
ax_plot.set_ylabel('Detection Rate')
ax_plot.set_title('Detection Rate vs Time to Landing')
ax_plot.tick_params(axis='x', rotation=45)

# --- Table (right) ---
# Prepare table data
table_data = [
    [str(bin_label), nom, edge] for bin_label, nom, edge in zip(
        combined_ttl_counts['ttl_bin'],
        combined_ttl_counts['Real-Nominal Count'],
        combined_ttl_counts['Real-Edge Count']
    )
]

# Add total row
total_nominal = combined_ttl_counts['Real-Nominal Count'].sum()
total_edge = combined_ttl_counts['Real-Edge Count'].sum()
table_data.append(["Total", total_nominal, total_edge])

# Hide table axes
ax_table.axis('off')

# Create the table
table = ax_table.table(
    cellText=table_data,
    colLabels=["TTL Bin", "Nominal", "Edge"],
    cellLoc='center',
    loc='center'
)

# Style table
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1.2, 1.4)

plt.tight_layout()
plt.show()

Infer ttl for synthetic if possible

# Failure Analysis

In [ ]:
print("--- Calculating and Visualizing False Negatives ---")

# Calculate and Print Stats
for dataset_type, config in DATASET_CONFIGS.items():
    df = data[dataset_type]['df']
    false_negatives_count = len(df[df['detected'] == False])
    total_count = len(df)
    dataset_name = config.get('name', dataset_type)
    print(f"({dataset_name}): {false_negatives_count} / {total_count} false negatives")

In [ ]:
# Set up a subplot grid to display samples from each dataset
num_datasets = len(DATASET_CONFIGS)
num_samples = 3
fig, axs = plt.subplots(num_datasets, num_samples, figsize=(num_samples * 5, num_datasets * 5))

# Ensure axs is always a 2D array, even if there's only one dataset
if num_datasets == 1:
    axs = axs.reshape(1, -1)

# Loop through each dataset to find and visualize its false negatives
for row_idx, (dataset_type, config) in enumerate(DATASET_CONFIGS.items()):

    # Get the necessary data for this dataset from our central dictionaries
    df = data[dataset_type]['df']
    gt_data = data[dataset_type]['gt']
    image_folder = config['image_folder']
    display_name = config['name']

    # Filter the DataFrame to get only the rows corresponding to false negatives
    df_fn = df[df['detected'] == False]

    # If there are no false negatives for this dataset, skip to the next one
    if df_fn.empty:
        print(f"\nNo false negatives to visualize for {display_name}.")
        # Turn off the axes for this row of the plot
        for col_idx in range(num_samples):
            axs[row_idx, col_idx].axis('off')
        continue

    print(f"\n📸 Visualizing False Negatives for: {display_name}")
    # Take a random, sample of the false negative images to display
    sample_rows = df_fn.sample(min(num_samples, len(df_fn)), random_state=42)

    # Loop through the sampled images and draw them on the plot
    for col_idx, (_, row) in enumerate(sample_rows.iterrows()):
        img_name = row['image_name']
        orig_w, orig_h = row['width'], row['height']

        # Get the image path and ground truth boxes for this image
        img_path = os.path.join(image_folder, img_name)
        gt_boxes = gt_data.get(img_name, [])

        # Select the correct subplot for this image
        ax = axs[row_idx, col_idx]

        # Call the main drawing function to display the image and its ground truth
        draw_predictions_with_gt(
            img_path=img_path,
            gt_boxes_orig=gt_boxes,
            pred_boxes=[],  # Pass empty lists for predictions, as these are FNs
            pred_scores=[],
            orig_w=orig_w,
            orig_h=orig_h,
            ax=ax,
            dataset_name=display_name
        )

# Clean up the plot by hiding any subplots that were not used
for row in axs:
    for ax in row:
        if not ax.has_data():
            ax.axis('off')

plt.tight_layout()
plt.show()

## Images with multiple detections

In [ ]:
print("--- Calculating and Visualizing Multiple Detections ---")

all_multi_detections = []

# Loop through datasets to get all multi-detection instances
for dataset_type, config in DATASET_CONFIGS.items():
    outputs = data[dataset_type]['outputs']

    multi_list, count = get_multiple_detections(outputs, score_thresh=0.90, detailed=True)

    print(f"multiple detections in ({config['name']}): {count}")

    # Tag each result with its dataset_type and add to our master list
    for img_name, boxes, scores in multi_list:
        all_multi_detections.append((img_name, boxes, scores, dataset_type))

In [ ]:
if not all_multi_detections:
    print("\nNo instances of multiple detections found to visualize.")
else:
    num_images = min(len(all_multi_detections), 9)
    cols = 3
    rows = (num_images + cols - 1) // cols
    fig, axs = plt.subplots(rows, cols, figsize=(cols * 5, rows * 5))
    axs = axs.flatten()

    # Sample from the combined list
    sampled_detections = random.sample(all_multi_detections, num_images)

    # Plotting loop using the master list and central data structure
    for idx, (img_name, pred_boxes, pred_scores, dataset_tag) in enumerate(sampled_detections):

        # Get all metadata directly
        img_name_cleaned = os.path.basename(img_name)
        config = DATASET_CONFIGS[dataset_tag]
        df = data[dataset_tag]['df']
        gt_data = data[dataset_tag]['gt']

        row_match = df[df["image_name"] == img_name_cleaned]

        if row_match.empty:
            print(f"[!] Metadata not found for {img_name_cleaned} in {dataset_tag}")
            continue

        row = row_match.iloc[0]
        orig_w, orig_h = row["width"], row["height"]
        img_path = os.path.join(config['image_folder'], img_name_cleaned)
        gt_boxes = gt_data.get(img_name_cleaned, [])

        draw_predictions_with_gt(img_path, gt_boxes, pred_boxes, pred_scores, orig_w, orig_h, axs[idx], dataset_name=config['name'])

    # Hide unused subplots
    for j in range(idx + 1, len(axs)):
        axs[j].axis("off")

    plt.tight_layout()
    plt.show()

## False Negatives



In [ ]:
print("--- Running Evaluation on All Datasets ---")

all_results = {}
for dataset_type, config in DATASET_CONFIGS.items():
    print(f"\nEvaluating {config['name']}...")
    result = evaluate_dataset(dataset_type, score_threshold=0.90, iou_threshold=0.3)
    all_results[config['name']] = result

print("\n--- Evaluation Summary ---")
summary_df = pd.DataFrame.from_dict(all_results, orient='index')
# Optional: format columns for better display
summary_df[['precision', 'recall', 'f1_score']] = summary_df[['precision', 'recall', 'f1_score']].map('{:.3f}'.format)
print(summary_df)

In [ ]:
all_per_image_results = []
for dataset_type in DATASET_CONFIGS:
    results = evaluate_dataset(dataset_type, return_detailed=True)
    all_per_image_results.extend(results)

print("Per-image evaluation results obtained for all datasets.")

In [ ]:
# Identify all images containing at least one False Negative
# We use the 'all_per_image_results' list from our detailed evaluation.
fn_images = set()
for result in all_per_image_results:
    if any(result['is_gt_box_fn']):
        fn_images.add(result['image_name'])

print(f"Found {len(fn_images)} unique images containing at least one False Negative.")


# Prepare a unified DataFrame for analysis
# Concatenate all individual dataframes into one
all_dfs_list = [data[dtype]['df'] for dtype in DATASET_CONFIGS]
all_df = pd.concat(all_dfs_list, ignore_index=True)

# Add a boolean column to flag the images that have False Negatives
all_df['has_false_negative'] = all_df['image_name'].isin(fn_images)


# Create a plotting function to compare distributions

def plot_error_rate_by_feature(df, feature_col, error_col, num_bins=10, log_bins=False):
    """
    Calculates and plots the error rate as a function of a binned feature.
    (This version has corrected label formatting for float values)
    """
    df_copy = df.copy()
    if log_bins:
        min_val = df_copy[df_copy[feature_col] > 0][feature_col].min()
        max_val = df_copy[feature_col].max()
        bins = np.logspace(np.log10(min_val), np.log10(max_val), num_bins + 1)
    else:
        bins = num_bins

    df_copy['feature_bin'] = pd.cut(df_copy[feature_col], bins=bins, include_lowest=True, duplicates='drop')

    total_counts = df_copy.groupby('feature_bin', observed=False).size()
    error_counts = df_copy[df_copy[error_col] == True].groupby('feature_bin', observed=False).size()

    error_rate = (error_counts / total_counts).fillna(0) * 100
    error_rate_df = error_rate.reset_index(name='error_rate')

    plt.figure(figsize=(14, 7))
    ax = sns.barplot(data=error_rate_df, x='feature_bin', y='error_rate', color='#4c72b0')

    # Format the labels as floats with 2 decimal places instead of integers
    if log_bins:
        new_labels = [f"{int(b.left)}-{int(b.right)}" for b in error_rate_df['feature_bin']]
    else:
        new_labels = [f"{b.left:.2f}-{b.right:.2f}" for b in error_rate_df['feature_bin']]

    ax.set_xticks(ax.get_xticks())
    ax.set_xticklabels(new_labels, rotation=45, ha='right')

    plt.xlabel(f"{feature_col.replace('_', ' ').title()} Bins")
    plt.ylabel(f"{error_col.replace('has_', '').replace('_', ' ').title()} Rate (%)")
    plt.title(f"Model Failure Rate vs. {feature_col.replace('_', ' ').title()}", fontsize=16)

    for i, p in enumerate(ax.patches):
        total_in_bin = total_counts.iloc[i]
        rate_text = f'{p.get_height():.1f}%'
        count_text = f'n={total_in_bin}'

        ax.text(p.get_x() + p.get_width() / 2., p.get_height(), rate_text,
                fontsize=11, color='black', ha='center', va='bottom')
        ax.text(p.get_x() + p.get_width() / 2., p.get_height()/2, count_text,
                fontsize=9, color='white', ha='center', va='center', weight='bold')

    plt.tight_layout()
    plt.show()

In [ ]:
# Analyze and Plot 'runway_area'
print("\n--- Analysis of Runway Area ---")
# Use .describe() to get a statistical summary for both groups
with pd.option_context('display.max_rows', None, 'display.max_columns', None,'display.max_colwidth', None):
    display(all_df.groupby('has_false_negative')['runway_area'].describe())

plot_error_rate_by_feature(
    df=all_df,
    feature_col='runway_area',
    error_col='has_false_negative',
    num_bins=8, # You can adjust the number of bins
    log_bins=True
)

In [ ]:
# Analyze and Plot 'slant_distance' (for synthetic data)
print("\n--- Analysis of Slant Distance (Synthetic Dataset) ---")
# Slant distance only exists for the synthetic dataset
synth_df = all_df[all_df['original_dataset'] == 'LARD_test_synth'].copy()
synth_df['slant_distance'] = pd.to_numeric(synth_df['slant_distance'], errors='coerce')
synth_df.dropna(subset=['slant_distance'], inplace=True)

# Call the plotting function for the 'slant_distance' feature
# We set log_bins=False because this feature is not skewed over many orders of magnitude
plot_error_rate_by_feature(
    df=synth_df,
    feature_col='slant_distance',
    error_col='has_false_negative',
    num_bins=8,
    log_bins=False
)

In [ ]:
# Filter for the specific outlier cases
all_df['slant_distance'] = pd.to_numeric(all_df['slant_distance'], errors='coerce')

fn_low_distance_df = all_df[
    (all_df['original_dataset'] == 'LARD_test_synth') &
    (all_df['has_false_negative'] == True) &
    (all_df['slant_distance'] < 1.5)
].copy() # Use .copy() to avoid SettingWithCopyWarning

print(f"Found {len(fn_low_distance_df)} False Negative images with slant distance < 1.5 km.")


# Visualize a sample of these outlier images
if not fn_low_distance_df.empty:
    num_samples = min(len(fn_low_distance_df), 9)
    samples_to_viz = fn_low_distance_df.sample(num_samples, random_state=42)

    cols = 3
    rows = (num_samples + cols - 1) // cols
    fig, axs = plt.subplots(rows, cols, figsize=(cols * 5, rows * 5))
    axs = axs.flatten()

    print(f"\n--- Visualizing {num_samples} sample outlier False Negatives ---")

    for idx, (_, row) in enumerate(samples_to_viz.iterrows()):
        # Get all necessary info directly from the sampled row and our central data
        img_name = row['image_name']
        slant_dist = row['slant_distance']
        orig_w, orig_h = row['width'], row['height']
        dataset_type = 'synthetic' # We know this from our filter

        # Retrieve data from our central dictionaries
        config = DATASET_CONFIGS[dataset_type]
        img_path = os.path.join(config['image_folder'], img_name)
        gt_boxes = data[dataset_type]['gt'].get(img_name, [])
        preds = data[dataset_type]['outputs'].get(img_name, {})
        pred_boxes = preds.get('boxes', [])
        pred_scores = preds.get('scores', [])

        # Call drawing function
        ax = axs[idx]
        draw_predictions_with_gt(
            img_path=img_path,
            gt_boxes_orig=gt_boxes,
            pred_boxes=pred_boxes,
            pred_scores=pred_scores,
            orig_w=orig_w,
            orig_h=orig_h,
            ax=ax,
            dataset_name=config['name']
        )
        ax.set_title(f"{img_name}\nSlant Distance: {slant_dist:.2f} km", fontsize=9)

    # Hide any unused subplots
    for j in range(idx + 1, len(axs)):
        axs[j].axis('off')

    plt.tight_layout()
    plt.show()

else:
    print("\nNo False Negative images found matching the specified criteria.")

## False Positives

In [ ]:
print("--- Analyzing Images with at Least One False Positive ---")

# Initialize everything we need
images_with_fp = [] # The list of detailed results for images with FPs
fp_images_nominal = set()
fp_images_edge = set()
fp_images_synth = set()

# Iterate through all the detailed results
for result in all_per_image_results:
    # Check for the False Positive condition
    if False in result['is_predicted_box_tp']:

        # Add the detailed result to our main list for later use
        images_with_fp.append(result)

        # Add the image name to the correct set for counting
        image_name = result['image_name']
        dataset_name = result['dataset_name']

        if dataset_name == 'REAL_Nominal':
            fp_images_nominal.add(image_name)
        elif dataset_name == 'REAL_Edge_Cases':
            fp_images_edge.add(image_name)
        elif dataset_name == 'LARD_test_synth':
            fp_images_synth.add(image_name)

# Print the breakdown of counts
print(f"Real-Nominal Dataset: {len(fp_images_nominal)} images")
print(f"Real-Edge Dataset:    {len(fp_images_edge)} images")
print(f"Synthetic Dataset:    {len(fp_images_synth)} images")
print(f"\nTotal unique images with False Positives: {len(images_with_fp)}")


# Update the main DataFrame
all_fp_images = fp_images_nominal | fp_images_edge | fp_images_synth
all_df['has_false_positive'] = all_df['image_name'].isin(all_fp_images)
print("DataFrame 'all_df' has been updated with the 'has_false_positive' column.")

In [ ]:
def draw_detailed_predictions(ax, img_path, gt_boxes, result_dict, target_size=1024):
    """
    Draws ground truth boxes and color-coded model predictions on an image.
    - Ground Truth: Green
    - True Positives (TP): Blue
    - False Positives (FP): Red
    """
    # Load and resize the image for display
    img = Image.open(img_path).convert("RGB")
    orig_w, orig_h = img.size
    img = img.resize((target_size, target_size))
    ax.imshow(img)
    ax.axis('off')

    # Scale and draw the ground truth boxes in green
    scaled_gt_boxes = scale_boxes(gt_boxes, orig_w, orig_h, target_size, target_size)
    for box in scaled_gt_boxes:
        x1, y1, x2, y2 = box
        rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2, edgecolor='lime', facecolor='none')
        ax.add_patch(rect)

    # Get the model's predictions and their corresponding TP/FP flags
    pred_boxes = result_dict['predicted_boxes']
    is_tp_flags = result_dict['is_predicted_box_tp']

    # Loop through each prediction and draw it with the correct color
    for box, is_tp in zip(pred_boxes, is_tp_flags):
        # Set color to blue for True Positives, red for False Positives
        color = 'blue' if is_tp else 'red'
        x1, y1, x2, y2 = box
        rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2, linestyle='--', edgecolor=color, facecolor='none')
        ax.add_patch(rect)

In [ ]:
# Filter the detailed results to find images that have both a correct detection (TP)
# and an incorrect detection (FP).
images_with_tp_and_fp = []
for result in all_per_image_results:
    has_tp = True in result['is_predicted_box_tp']
    has_fp = False in result['is_predicted_box_tp']
    if has_tp and has_fp:
        images_with_tp_and_fp.append(result)

print(f"Found {len(images_with_tp_and_fp)} unique images containing both True Positives and False Positives.")


# Visualize a random sample of these images if any were found
if images_with_tp_and_fp:
    # Set up the visualization grid
    num_samples = min(len(images_with_tp_and_fp), 9)
    samples_to_viz = random.sample(images_with_tp_and_fp, num_samples)

    cols = 3
    rows = (num_samples + cols - 1) // cols
    fig, axs = plt.subplots(rows, cols, figsize=(cols * 6, rows * 6))
    axs = axs.flatten()

    print(f"\n--- Visualizing {num_samples} sample images with both TPs and FPs ---")
    print("Green = Ground Truth | Blue = True Positive | Red = False Positive")

    # Loop through the sampled images and plot them
    for idx, result in enumerate(samples_to_viz):
        img_name = result['image_name']
        dataset_name_from_result = result['dataset_name']

        # Find the short dataset key (e.g., 'nominal') from its full name
        dataset_type = next((key for key, conf in DATASET_CONFIGS.items() if conf['name'] == dataset_name_from_result), None)

        if not dataset_type:
            axs[idx].axis('off')
            continue

        # Get the necessary image path and ground truth data
        config = DATASET_CONFIGS[dataset_type]
        img_path = os.path.join(config['image_folder'], img_name)
        gt_boxes = data[dataset_type]['gt'].get(img_name, [])

        # Select the correct subplot and draw the image with annotations
        ax = axs[idx]
        draw_detailed_predictions(ax, img_path, gt_boxes, result)
        ax.set_title(f"{img_name}\n{dataset_name_from_result}", fontsize=9)

        # Manually create a legend for each subplot for clarity
        handles = [patches.Patch(color='lime', label='Ground Truth'),
                   patches.Patch(color='blue', label='True Positive'),
                   patches.Patch(color='red', label='False Positive')]
        ax.legend(handles=handles, fontsize='small')


    # Hide any unused subplots in the grid
    for j in range(idx + 1, len(axs)):
        axs[j].axis('off')

    plt.tight_layout()
    plt.show()

else:
    print("\nNo images found containing both TPs and FPs.")

### Analysis of FP

In [ ]:
print("\n--- Analysis of Runway Area for False Positives ---")

with pd.option_context('display.max_rows', None, 'display.max_columns', None,'display.max_colwidth', None):
    display(all_df.groupby('has_false_positive')['runway_area'].describe())

# We use 'plot_error_rate_by_feature' to see how the FP rate changes with area.
plot_error_rate_by_feature(
    df=all_df,
    feature_col='runway_area',
    error_col='has_false_positive',
    num_bins=8,
    log_bins=True
)

In [ ]:
# Analyze and Plot 'slant_distance' (for synthetic data) ---
print("\n--- Analysis of Slant Distance for False Positives (Synthetic Dataset) ---")
# Slant distance only exists for the synthetic dataset
synth_df_fp = all_df[all_df['original_dataset'] == 'LARD_test_synth'].copy()
if 'slant_distance' in synth_df_fp.columns:
    # Add the FP flag specifically to this subset
    synth_df_fp['has_false_positive'] = synth_df_fp['image_name'].isin(all_fp_images)
    synth_df_fp['slant_distance'] = pd.to_numeric(synth_df_fp['slant_distance'], errors='coerce')
    synth_df_fp.dropna(subset=['slant_distance'], inplace=True)

    display(synth_df_fp.groupby('has_false_positive')['slant_distance'].describe())
    plot_error_rate_by_feature(
        df=synth_df_fp,
        feature_col='slant_distance',
        error_col='has_false_positive',
        num_bins=8,
        log_bins=False
    )

In [ ]:
# Select a random sample of images to visualize
num_samples = min(len(images_with_fp), 21)
samples_for_viz = random.sample(images_with_fp, num_samples)

# Create figure for plotting
cols = 3
rows = (num_samples + cols - 1) // cols
fig, axs = plt.subplots(rows, cols, figsize=(cols * 6, rows * 6))
axs = axs.flatten()

print(f"\n--- Visualizing {num_samples} sample images with False Positives ---")

# Iterate through the samples and plot
for idx, result in enumerate(samples_for_viz):
    # Get image name and dataset name directly from our rich result object
    img_name = result['image_name']
    dataset_name_from_result = result['dataset_name']

    # Find the corresponding 'dataset_type' key (e.g., 'nominal')
    dataset_type = next((key for key, conf in DATASET_CONFIGS.items() if conf['name'] == dataset_name_from_result), None)

    if not dataset_type:
        axs[idx].axis('off')
        continue

    # Get all other data directly from our central data structures
    config = DATASET_CONFIGS[dataset_type]
    df = data[dataset_type]['df']
    row_match = df[df['image_name'] == img_name]

    if row_match.empty:
        axs[idx].axis('off')
        continue

    row = row_match.iloc[0]
    orig_w, orig_h = row['width'], row['height']
    img_path = os.path.join(config['image_folder'], img_name)
    gt_boxes = data[dataset_type]['gt'].get(img_name, [])

    # Get the original predictions for this image to draw them all
    preds = data[dataset_type]['outputs'].get(img_name, {})
    pred_boxes = preds.get('boxes', [])
    pred_scores = preds.get('scores', [])

    # Call drawing function
    ax = axs[idx]
    draw_predictions_with_gt(
        img_path=img_path,
        gt_boxes_orig=gt_boxes,
        pred_boxes=pred_boxes,
        pred_scores=pred_scores,
        orig_w=orig_w,
        orig_h=orig_h,
        ax=ax,
        dataset_name=config['name']
    )
    ax.set_title(f"{img_name}\nDataset: {config['name']}", fontsize=9)


# Hide any unused subplots
for j in range(idx + 1, len(axs)):
    axs[j].axis('off')

plt.tight_layout()
plt.show()

## Edge case: Image with no TP but FP (also FN)

In [ ]:
def filter_images_preds_no_tp_fn(per_image_results):
    """
    Filters for a specific failure case: images where the model made predictions (FPs),
    missed the ground truth (FNs), but got zero correct detections (no TPs).
    """
    filtered_list = []
    # Go through each image's detailed results
    for img_result in per_image_results:
        # Check for at least one prediction
        has_predictions = len(img_result['predicted_boxes']) > 0
        # Check that none of the predictions were True Positives
        has_no_tp = not any(img_result['is_predicted_box_tp'])
        # Check that there was at least one ground truth that was missed
        has_fn_gt = any(img_result['is_gt_box_fn'])

        # If all conditions for this specific failure case are met, add it to our list
        if has_predictions and has_no_tp and has_fn_gt:
            filtered_list.append(img_result)

    return filtered_list


# Apply the filter function once to the combined list of all image results
filtered_results = filter_images_preds_no_tp_fn(all_per_image_results)


# Summarize the results dynamically using a Counter to count failures per dataset
dataset_counts = Counter(result['dataset_name'] for result in filtered_results)

# Print the breakdown of counts for each dataset
print(f"\nNumber of images with predictions, no TP, and FN (Nominal): {dataset_counts.get('REAL_Nominal', 0)}")
print(f"Number of images with predictions, no TP, and FN (Edge): {dataset_counts.get('REAL_Edge_Cases', 0)}")
print(f"Number of images with predictions, no TP, and FN (Synthetic): {dataset_counts.get('LARD_test_synth', 0)}")
print(f"Total number of unique images with this failure case: {len(filtered_results)}")


# Display a few sample image names from the filtered list
if filtered_results:
    print("\nSample images with this failure case:")
    for result in filtered_results[:5]:
        print(f"- {result['image_name']} (from {result['dataset_name']})")
else:
    print("\nNo images found with predictions, no TP, and FN.")

In [ ]:
num_samples = min(len(filtered_results), 9)

samples_for_viz = random.sample(filtered_results, num_samples)

# 2. Create figure for plotting
cols = 3
rows = (num_samples + cols - 1) // cols
fig, axs = plt.subplots(rows, cols, figsize=(cols * 5, rows * 5))
axs = axs.flatten()

print(f"--- Visualizing {num_samples} sample images with FP and FN ---")

# 3. Iterate through the samples and plot
for idx, result in enumerate(samples_for_viz):
    # Get image name and dataset tag
    img_name = result['image_name']
    dataset_name_from_result = result['dataset_name'] # e.g., 'REAL_Nominal'

    # Find the corresponding 'dataset_type' key (e.g., 'nominal')
    dataset_type = None
    for key, conf in DATASET_CONFIGS.items():
        if conf['name'] == dataset_name_from_result:
            dataset_type = key
            break

    if not dataset_type:
        axs[idx].axis('off')
        continue

    # Get all other data directly from our central data structures
    config = DATASET_CONFIGS[dataset_type]
    df = data[dataset_type]['df']
    row_match = df[df['image_name'] == img_name]

    if row_match.empty:
        axs[idx].axis('off')
        continue

    row = row_match.iloc[0]
    orig_w, orig_h = row['width'], row['height']
    img_path = os.path.join(config['image_folder'], img_name)
    gt_boxes = data[dataset_type]['gt'].get(img_name, [])

    # Get the original predictions for this image
    preds = data[dataset_type]['outputs'].get(img_name, {})
    pred_boxes = preds.get('boxes', [])
    pred_scores = preds.get('scores', [])

    # Call drawing function
    ax = axs[idx]
    draw_predictions_with_gt(
        img_path=img_path,
        gt_boxes_orig=gt_boxes,
        pred_boxes=pred_boxes,
        pred_scores=pred_scores,
        orig_w=orig_w,
        orig_h=orig_h,
        ax=ax,
        dataset_name=config['name']
    )
    ax.set_title(f"{img_name}\nDataset: {config['name']}", fontsize=8)

# Hide any unused subplots
for j in range(idx + 1, len(axs)):
    axs[j].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
def find_duplicate_detections(detailed_results, iou_threshold=0.3):
    """
    Precisely identifies images where multiple predictions match the same
    ground truth box.
    """
    duplicate_cases = []

    for result in detailed_results:
        gt_boxes_orig = result.get('ground_truth_boxes', [])
        pred_boxes = result.get('predicted_boxes', [])

        if not gt_boxes_orig:
            continue

        img_name = result['image_name']
        row_match = all_df[all_df['image_name'] == img_name]
        if row_match.empty:
            continue
        orig_w, orig_h = row_match.iloc[0]['width'], row_match.iloc[0]['height']

        # Call scale_boxes
        scaled_gt_boxes = scale_boxes(gt_boxes_orig, orig_w, orig_h, 1024, 1024)

        is_duplicate = False
        for gt_box in scaled_gt_boxes:
            matches = 0
            for pred_box in pred_boxes:
                if compute_iou(pred_box, gt_box) >= iou_threshold:
                    matches += 1

            if matches > 1:
                is_duplicate = True
                break

        if is_duplicate:
            duplicate_cases.append(result)

    return duplicate_cases

duplicate_detection_cases = find_duplicate_detections(all_per_image_results, iou_threshold=0.3)

print(f"Found {len(duplicate_detection_cases)} images with precise duplicate detections.")

In [ ]:
if duplicate_detection_cases:
    num_samples = min(len(duplicate_detection_cases), 9)
    samples_to_viz = random.sample(duplicate_detection_cases, num_samples)

    cols = 3
    rows = (num_samples + cols - 1) // cols
    fig, axs = plt.subplots(rows, cols, figsize=(cols * 6, rows * 6))
    axs = axs.flatten()

    print(f"\n--- Visualizing {num_samples} sample images with duplicate detections ---")
    print("Green = Ground Truth | Blue = Best Prediction (TP) | Red = Duplicate Prediction (FP)")

    for idx, result in enumerate(samples_to_viz):
        # (The rest of your existing visualization code goes here and will work perfectly)
        img_name = result['image_name']
        dataset_name_from_result = result['dataset_name']

        dataset_type = next((key for key, conf in DATASET_CONFIGS.items() if conf['name'] == dataset_name_from_result), None)

        if not dataset_type:
            axs[idx].axis('off')
            continue

        config = DATASET_CONFIGS[dataset_type]
        img_path = os.path.join(config['image_folder'], img_name)
        gt_boxes = data[dataset_type]['gt'].get(img_name, [])

        ax = axs[idx]
        draw_detailed_predictions(ax, img_path, gt_boxes, result)
        ax.set_title(f"{img_name}\n{dataset_name_from_result}", fontsize=9)

        handles = [patches.Patch(color='lime', label='Ground Truth'),
                   patches.Patch(color='blue', label='True Positive'),
                   patches.Patch(color='red', label='False Positive')]
        ax.legend(handles=handles, fontsize='small')

    for j in range(idx + 1, len(axs)):
        axs[j].axis('off')

    plt.tight_layout()
    plt.show()

else:
    print("\nNo images found matching this specific failure case.")

In [ ]:
def save_image_with_boxes(result, save_dir):
    """
    Creates and saves an annotated image file for a given detection result.
    It draws GT, TP, and FP boxes with dynamic line widths for clarity.
    """
    try:
        # Get all the necessary metadata for the image from our central data stores
        img_name = result['image_name']
        dataset_name = result['dataset_name']
        dataset_type = next((key for key, conf in DATASET_CONFIGS.items() if conf['name'] == dataset_name), None)

        # If we can't find the dataset, skip this image
        if not dataset_type: return

        config = DATASET_CONFIGS[dataset_type]
        img_path = os.path.join(config['image_folder'], img_name)
        gt_boxes_orig = result.get('ground_truth_boxes', [])

        # Set up a new matplotlib figure for this image
        fig, ax = plt.subplots(figsize=(12, 12))
        img = Image.open(img_path).convert("RGB")
        orig_w, orig_h = img.size
        ax.imshow(img)
        ax.axis('off')

        # Draw the ground truth boxes in green on the original image
        for box in gt_boxes_orig:
            x1, y1, x2, y2 = box
            w, h = x2 - x1, y2 - y1
            # Calculate a dynamic line width based on the box size
            linewidth = max(1, min(5, (w**2 + h**2)**0.5 / 150))
            rect = patches.Rectangle((x1, y1), w, h, linewidth=linewidth, edgecolor='lime', facecolor='none')
            ax.add_patch(rect)

        # Draw the model's prediction boxes (Blue for TP, Red for FP)
        pred_boxes_model_space = result.get('predicted_boxes', [])
        # Scale the predictions from model space (1024x1024) back to original image coordinates
        pred_boxes_orig_space = scale_boxes(pred_boxes_model_space, orig_w=1024, orig_h=1024, target_w=orig_w, target_h=orig_h)
        is_tp_flags = result.get('is_predicted_box_tp', [])

        for box, is_tp in zip(pred_boxes_orig_space, is_tp_flags):
            color = 'blue' if is_tp else 'red'
            x1, y1, x2, y2 = box
            w, h = x2 - x1, y2 - y1
            # Apply the same dynamic line width logic to the prediction boxes
            linewidth = max(1, min(4, (w**2 + h**2)**0.5 / 150))
            rect = patches.Rectangle((x1, y1), w, h, linewidth=linewidth, linestyle='--', edgecolor=color, facecolor='none')
            ax.add_patch(rect)

        # Add title, legend, and save the final annotated image
        ax.set_title(f"{img_name}\n{dataset_name}", fontsize=12)
        handles = [patches.Patch(color='lime', label='Ground Truth'),
                   patches.Patch(color='blue', label='True Positive'),
                   patches.Patch(color='red', label='False Positive')]
        ax.legend(handles=handles, fontsize='medium')

        save_path = os.path.join(save_dir, img_name)
        plt.savefig(save_path, bbox_inches='tight', pad_inches=0.1)
        # Close the figure to free up memory, which is important when processing many images
        plt.close(fig)

    except Exception as e:
        # Catch any errors during processing to prevent the whole script from crashing
        print(f"Failed to process image {img_name}. Error: {e}")
        plt.close('all') # Make sure to close any lingering plots on error

# Overlapping BBoxes

In [ ]:
# Define the specific list of image filenames we want to visualize as examples
image_filenames_to_visualize = [
    '4s4zQFW4g7o_070.png',
    'i8RhaLAUZ5g_074.png',
    'LSZH_34_35_25.jpeg',
    'LWSK_16_35_08.jpeg',
    'rjWG3WRM5yk_067.png'
]

# Find the detailed result objects for these specific images from our master results list
results_to_visualize = []
for result in all_per_image_results:
    if result['image_name'] in image_filenames_to_visualize:
        results_to_visualize.append(result)

print(f"Found {len(results_to_visualize)} of the specified images for visualization.")


# Create the plot if we found any of the requested images
if results_to_visualize:
    # Set up the plot grid to display the images
    num_samples = len(results_to_visualize)
    cols = 3
    rows = (num_samples + cols - 1) // cols
    fig, axs = plt.subplots(rows, cols, figsize=(cols * 6, rows * 5))
    axs = axs.flatten()

    print("\n--- Visualizing Specific Overlapping Detection Cases ---")
    print("Green = Ground Truth | Blue = True Positive | Red = False Positive (Overlap)")

    # Loop through our selected images and plot each one on a subplot
    for idx, result in enumerate(results_to_visualize):
        img_name = result['image_name']
        dataset_name_from_result = result['dataset_name']

        # Find the short dataset key (e.g., 'nominal') from its full name
        dataset_type = next((key for key, conf in DATASET_CONFIGS.items() if conf['name'] == dataset_name_from_result), None)

        if not dataset_type:
            axs[idx].axis('off')
            continue

        # Get the image path and ground truth boxes
        config = DATASET_CONFIGS[dataset_type]
        img_path = os.path.join(config['image_folder'], img_name)
        gt_boxes = data[dataset_type]['gt'].get(img_name, [])

        # Select the correct subplot and call our drawing function
        ax = axs[idx]
        draw_detailed_predictions(ax, img_path, gt_boxes, result)
        ax.set_title(f"{img_name}\n{dataset_name_from_result}", fontsize=9)

        # Add a legend to each subplot for clarity
        handles = [patches.Patch(color='lime', label='Ground Truth'),
                   patches.Patch(color='blue', label='True Positive'),
                   patches.Patch(color='red', label='False Positive')]
        ax.legend(handles=handles, fontsize='small')

    # Hide any empty subplots for a cleaner final image
    for j in range(num_samples, len(axs)):
        axs[j].axis('off')

    plt.tight_layout()
    plt.show()

else:
    print("\nCould not find the specified images in the results list.")

# Logos

In [ ]:
# Define the specific list of image filenames to visualize as examples
image_filenames_to_visualize = [
    'yL_zy-9SDsg_093.png',
    'yL_zy-9SDsg_089.png',
    'yL_zy-9SDsg_087.png',
    'LEMD_14L_35_21.jpeg',
    '86ghu0Dcaog_109.png'
]

# Find the detailed result objects for these specific images from our master results list
results_to_visualize = []
for result in all_per_image_results:
    if result['image_name'] in image_filenames_to_visualize:
        results_to_visualize.append(result)

print(f"Found {len(results_to_visualize)} of the specified images for visualization.")

# Create the plot if we found any of the requested images
if results_to_visualize:
    # Set up the plot grid to display the images
    num_samples = len(results_to_visualize)
    cols = 3
    rows = (num_samples + cols - 1) // cols
    fig, axs = plt.subplots(rows, cols, figsize=(cols * 6, rows * 5))
    axs = axs.flatten()

    print("\n--- Visualizing Specific 'Hallucination' Cases (Logos) ---")
    print("Green = Ground Truth | Red = False Positive")

    # Loop through our selected images and plot each one on a subplot
    for idx, result in enumerate(results_to_visualize):
        img_name = result['image_name']
        dataset_name_from_result = result['dataset_name']

        # Find the short dataset key (e.g., 'nominal') from its full name
        dataset_type = next((key for key, conf in DATASET_CONFIGS.items() if conf['name'] == dataset_name_from_result), None)

        if not dataset_type:
            axs[idx].axis('off')
            continue

        # Get the image path and ground truth boxes
        config = DATASET_CONFIGS[dataset_type]
        img_path = os.path.join(config['image_folder'], img_name)
        gt_boxes = data[dataset_type]['gt'].get(img_name, [])

        # Select the correct subplot and call our drawing function
        ax = axs[idx]
        draw_detailed_predictions(ax, img_path, gt_boxes, result)
        ax.set_title(f"{img_name}\n{dataset_name_from_result}", fontsize=9)

        # Add a legend to each subplot for clarity
        handles = [patches.Patch(color='lime', label='Ground Truth'),
                   patches.Patch(color='red', label='False Positive')]
        ax.legend(handles=handles, fontsize='small')

    # Hide any empty subplots for a cleaner final image
    for j in range(num_samples, len(axs)):
        axs[j].axis('off')

    plt.tight_layout()
    plt.show()

else:
    print("\nCould not find the specified images in the results list.")

# Extended

In [ ]:
# Define the specific list of image filenames to visualize as examples
image_filenames_to_visualize = [
    'CC-lSLLiueE_036.png',
    'fB33KKArerU_059.png',
    'JgKWRjH2O10_069.png',
    'KMIA_9_35_26.jpeg',
    'LEMD_32R_35_18.jpeg',
    'LWSK_16_35_08.jpeg'
]

# Find the detailed result objects for these specific images from our master results list
results_to_visualize = []
for result in all_per_image_results:
    if result['image_name'] in image_filenames_to_visualize:
        results_to_visualize.append(result)

print(f"Found {len(results_to_visualize)} of the specified images for visualization.")

# Create the plot if we found any of the requested images
if results_to_visualize:
    # Set up the plot grid to display the images
    num_samples = len(results_to_visualize)
    cols = 3
    rows = (num_samples + cols - 1) // cols
    fig, axs = plt.subplots(rows, cols, figsize=(cols * 6, rows * 5))
    axs = axs.flatten()

    print("\n--- Visualizing Incomplete Ground Truth Cases ---")
    print("Green = Ground Truth (Incomplete) | Blue/Red = Model Prediction (More Complete)")

    # Loop through our selected images and plot each one on a subplot
    for idx, result in enumerate(results_to_visualize):
        img_name = result['image_name']
        dataset_name_from_result = result['dataset_name']

        # Find the short dataset key (e.g., 'nominal') from its full name
        dataset_type = next((key for key, conf in DATASET_CONFIGS.items() if conf['name'] == dataset_name_from_result), None)

        if not dataset_type:
            axs[idx].axis('off')
            continue

        # Get the image path and ground truth boxes
        config = DATASET_CONFIGS[dataset_type]
        img_path = os.path.join(config['image_folder'], img_name)
        gt_boxes = data[dataset_type]['gt'].get(img_name, [])

        # Select the correct subplot and call our drawing function
        ax = axs[idx]
        draw_detailed_predictions(ax, img_path, gt_boxes, result)
        ax.set_title(f"{img_name}\n{dataset_name_from_result}", fontsize=9)

        # Add a legend to each subplot for clarity
        handles = [patches.Patch(color='lime', label='Ground Truth'),
                   patches.Patch(color='blue', label='True Positive'),
                   patches.Patch(color='red', label='False Positive')]
        ax.legend(handles=handles, fontsize='small')

    # Hide any empty subplots for a cleaner final image
    for j in range(num_samples, len(axs)):
        axs[j].axis('off')

    plt.tight_layout()
    plt.show()

else:
    print("\nCould not find the specified images in the results list.")

# control panel

In [ ]:
# Define the specific list of image filenames to visualize as examples of this failure case
image_filenames_to_visualize = [
    'sxzWZX_I9hE_120.png', 'sxzWZX_I9hE_121.png',
    'sxzWZX_I9hE_122.png', 'sqD0iHHDHqo_074.png',
    'pA2yimUN7VI_083.png', 'pA2yimUN7VI_084.png',
    'JMomkYzMYpA_047.png', 'JMomkYzMYpA_049.png'
]

# Find the detailed result objects for these specific images using a list comprehension
results_to_visualize = [res for res in all_per_image_results if res['image_name'] in image_filenames_to_visualize]

print(f"Found {len(results_to_visualize)} of the specified images for visualization.")

# Create the plot if we found any of the requested images
if results_to_visualize:
    # Set up the plot grid to display the images
    num_samples = len(results_to_visualize)
    cols = 4
    rows = (num_samples + cols - 1) // cols
    fig, axs = plt.subplots(rows, cols, figsize=(cols * 6, rows * 5))
    axs = axs.flatten()

    print("\n--- Visualizing 'Hallucination' Cases (Control Panels) ---")
    print("Red = False Positive")

    # Loop through our selected images and plot each one on a subplot
    for idx, result in enumerate(results_to_visualize):
        img_name = result['image_name']
        dataset_name_from_result = result['dataset_name']

        # Find the short dataset key (e.g., 'nominal') from its full name
        dataset_type = next((key for key, conf in DATASET_CONFIGS.items() if conf['name'] == dataset_name_from_result), None)

        if not dataset_type:
            axs[idx].axis('off')
            continue

        # Get the image path and ground truth boxes (which will be empty for these cases)
        config = DATASET_CONFIGS[dataset_type]
        img_path = os.path.join(config['image_folder'], img_name)
        gt_boxes = data[dataset_type]['gt'].get(img_name, [])

        # Select the correct subplot and call our drawing function
        ax = axs[idx]
        draw_detailed_predictions(ax, img_path, gt_boxes, result)
        ax.set_title(f"{img_name}\n{dataset_